## Import necessary python modules

In [13]:
import os
import napari
import tifffile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog
import bigfish
import bigfish.plot as plot
import bigfish.stack as stack
import bigfish.detection as detection
import bigfish.multistack as multistack
from pathlib import Path
from copy import deepcopy
from dask.array.image import imread as imr
from bigfish.detection.utils import get_object_radius_pixel
from buildReferenceSpot import buildReferenceSpotFromImages
from runBigfishDetection import getSpotAndClusters, saveSpotsNPZ, reorderZstack

In [14]:
from cellpose import models
from cellpose.utils import remove_edge_masks
from cellpose.io import imread, save_to_png, masks_flows_to_seg
# import liveQuant
from pathlib import Path

## Specify voxel and object size

In [15]:
voxelRadius = (600, 121, 121)
objectRadius = (600, 150, 150)

In [16]:
from scipy.optimize import curve_fit

def bi_exp(x, a, b, c, d):
    return (a * np.exp(-b * x)) + (c * np.exp(-d * x))

def trip_exp(x, a, b, c, d, e, f):
    return ((a * np.exp(-b * x)) + (c * np.exp(-d * x)) + (e * np.exp(-f * x)))

def getBleachCorrected(stackCell, model='bi'):
    axes = tuple([i for i in range(len(stackCell.shape))])
    I_mean = np.mean(stackCell, axis=axes[1:])
    timePoints = np.arange(stackCell.shape[0])
    
    if model=='bi':
        coeffsExp, _ = curve_fit(bi_exp, timePoints, I_mean, maxfev=50000)
        f_ = np.vectorize(bi_exp)(timePoints, *coeffsExp)
    elif model=='tri':
        coeffsExp, _ = curve_fit(trip_exp, timePoints, I_mean, maxfev=50000)
        f_ = np.vectorize(trip_exp)(timePoints, *coeffsExp)
    
    
    f = f_ / np.max(f_)
    f = f.reshape(-1, 1, 1, 1)
    imagesCorrected = (stackCell / f).astype(np.uint16)

    # calculate r squared
    residuals = I_mean - f_
    ss_res = np.sum(residuals ** 2)
    ss_tot = np.sum((I_mean - np.mean(I_mean)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)
    r_squared_exp = np.array(r_squared)
    return imagesCorrected, r_squared_exp, I_mean

In [17]:
def getImagesAndSpotList(sequenceCell, selectedThreshold, voxelRadius, objectRadius, sampling=10):
    images=[]    
    spots_list=[]
    MaxTimePoint = sequenceCell.shape[0]

    spot_radius_px = detection.get_object_radius_pixel(
        voxel_size_nm=voxelRadius, 
        object_radius_nm=objectRadius, 
        ndim=3)

    for t in range(1,MaxTimePoint,sampling):
        rna = np.array(sequenceCell[t])
        images.append(rna)

        # LoG filter
        rna_log = stack.log_filter(rna, sigma=spot_radius_px)

        # local maximum detection
        mask = detection.local_maximum_detection(rna_log, min_distance=spot_radius_px)

        # thresholding
        threshold = detection.automated_threshold_setting(rna_log, mask)
        spots_, _ = detection.spots_thresholding(rna_log, mask, float(selectedThreshold))
        spots_list.append(spots_)
    n=len(images)
    print("Total number of images : "+str(n))
    return images, spots_list, n

In [18]:
def choose_home_folder():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    file_path = filedialog.askdirectory(initialdir= "/", title='Please select a directory')  # Open file dialog

    root.destroy()  # Close the tkinter window
    return file_path


def getCellMask(cellNumber, labeldf, maskImageAll):
    cellNumber = int(cellNumber.split('_')[-1])
    labeldf[labeldf['label']==cellNumber]
    x00 = int(np.round(np.array(labeldf[labeldf['label']==cellNumber]['x'])[0]))
    y00 = int(np.round(np.array(labeldf[labeldf['label']==cellNumber]['y'])[0]))
    sizex = mipSequenceCell.shape[1]
    sizey = mipSequenceCell.shape[2]
    nminr_ = x00 - sizex//2
    nmaxr_ = x00 + sizex//2
    nminc_ = y00 - sizey//2
    nmaxc_ = y00 + sizey//2

    bxt = (nminc_, nmaxc_, nmaxc_, nminc_, nminc_)
    byt = (nminr_, nminr_, nmaxr_, nmaxr_, nminr_)

    maskCell = maskImageAll[nminr_:nmaxr_,nminc_:nmaxc_]==maskImageAll[x00, y00]
    maskCell = stack.dilation_filter(maskCell, kernel_shape='disk', kernel_size=3)
    return maskCell

def removeSpots(old_spots_list, maskCell):
    new_spots_list = []
    for i in range(len(old_spots_list)):
        new_spots_list
        mask_in = maskCell[old_spots_list[i][:, 1], old_spots_list[i][:, 2]]
        newSpots = np.array(pd.DataFrame(old_spots_list[i])[mask_in]) 
        new_spots_list.append(newSpots)
        if len(new_spots_list) == len(old_spots_list):
            print('all good')
    return new_spots_list



In [19]:
df = pd.DataFrame(columns=['imsq','cell_number','threshold','spots_found','max_intensity','no.frames','referenceSpot_Alpha','referenceSpot_gamma','beta','gamma','numberSpotmin'])

In [20]:
midentifier = 'cell_'
fidentifier = 'basal'

In [21]:

baseFolder0 = choose_home_folder()#homeFolder+nameKey+imsQ
print("Chosen home folder:", baseFolder0) # folder containg folder of movies
basefolders = [os.path.join(baseFolder0+'/'+i) for i in os.listdir(baseFolder0) if fidentifier in i and os.path.isdir(os.path.join(baseFolder0,i))]
basefolders.sort()
basefolders

Chosen home folder: C:/Users/uid-1204/Desktop/test2_bigFISHLIVE


['C:/Users/uid-1204/Desktop/test2_bigFISHLIVE/Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06']

## Specify Input and Output folders, identifiers

In [23]:
# homeFolder = '/media/rachel/9d56c1ff-e031-4e35-9f3c-fcd7d3e80033/Analysis/20230720/'
referenceSpot_Alpha=0.6
referenceSpot_gamma=3
bEta=1
gAmma=3
numberSpotmin=2
reorderStack = False
opbCorrection = False

for baseFolder in basefolders[:]:
    nameKey = Path(baseFolder).name

    imsQ = nameKey.split('_F')[-1]
    thresholdFile = pd.read_csv(os.path.join(baseFolder,'thresholds.csv'), index_col=0)

    for fieldNumber in range(len(thresholdFile)): #[np.where(thresholdFile['cell']=='cell_'+str(cellNumber))[0][0]]:#
        print(fieldNumber)
        print('cell number :', thresholdFile['cell'][fieldNumber])
        cellNumber = thresholdFile['cell'][fieldNumber]
        path_input = os.path.join(baseFolder,cellNumber)
        pathToTimeFramesCell = os.path.join(path_input,'*.tif')

        sequenceCell = imr(pathToTimeFramesCell)
        mipSequenceCell = np.max(sequenceCell, axis=1)
        MaxTimePoint = sequenceCell.shape[0]
        selectedThreshold = float(thresholdFile['threshold'][fieldNumber])
        print('selected threshold : ', selectedThreshold)
        print('path_in/out:', path_input)

        labeldf = pd.read_pickle(os.path.join(baseFolder,'LabelDF.pkl'))
        tProj = np.max(mipSequenceCell, axis=0)
        maskpath = os.path.join(Path(baseFolder).parent, 'tProjections','T_MAX_'+Path(baseFolder).name.replace('.','_')+'_cp_masks.png')
        maskImageAll = imread(maskpath)
        maskCell = getCellMask(cellNumber, labeldf, maskImageAll)

        if selectedThreshold!=0:
            if opbCorrection==True:
                sequenceCellInput, r_bi, I_mean = getBleachCorrected(sequenceCell, model='bi')
            else:
                sequenceCellInput = sequenceCell
            ## Get a list of spots detected using the threshold specified in previous step
            images, spots_list, no_frames = getImagesAndSpotList(sequenceCellInput, 
                                                                 selectedThreshold, 
                                                                 voxelRadius, 
                                                                 objectRadius, sampling=5)
            
#             ## Remove unwanted spots
#             try:
#                 spots_list = removeSpots(spots_list, maskCell)
#             except IndexError:
#                 pass
#             try:
#                 plt.figure()
#                 plt.imshow(tProj)
#                 plt.imshow(maskCell, alpha=0.2)
#             except ValueError:
#                 pass
#     #             plt.figure()
#     #             plt.imshow(mipSequenceCell[0])
#     #             plt.imshow(maskCell, alpha=0.2)
            
            ## Build reference spot
            try:
                reference_spot, spots_found, max_intensity = buildReferenceSpotFromImages(images, spots_list, 
                                                                                      alpha=referenceSpot_Alpha, 
                                                                                      gamma=referenceSpot_gamma, voxelSize=voxelRadius, 
                                                                                          objectSize=objectRadius)
                refSpot = deepcopy(reference_spot)

                ## Perform spot and cluster detection for all frames
                spotsFrame, clustersFrames, ThresholdFrames = getSpotAndClusters(sequenceCell, 
                                                                             reference_spot, 
                                                                             cellnumber=cellNumber, 
                                                                             startTime=0,
                                                                             stopTime=MaxTimePoint, 
                                                                             thresholdManual=selectedThreshold, 
                                                                             beta=bEta, 
                                                                             gamma=gAmma,
                                                                             numberOfSpots=numberSpotmin,
                                                                             radiusCluster=400, 
                                                                             voxelSize=voxelRadius, 
                                                                             objectSize=objectRadius,
                                                                             reorder=reorderStack,
                                                                             extensionMov='.tif',
                                                                             showProgress=False)

                ## Save spots clusters and reference spot
                saveSpotsNPZ(np.array(spotsFrame, dtype=object), 
                             np.array(clustersFrames, dtype=object), 
                             np.array(ThresholdFrames, dtype=object), 
                             cellNumber, 
                             path_input, 
                             reference_spot,
                             threshold = selectedThreshold,
                             outputstring = '_spots_and_clusters_27_May'+str(referenceSpot_Alpha))

                df2 = pd.DataFrame(np.array([imsQ,cellNumber,selectedThreshold,
                                             spots_found,max_intensity,no_frames,referenceSpot_Alpha,
                                             referenceSpot_gamma,bEta,gAmma,numberSpotmin]).reshape(1,11), columns=['imsq','cell_number','threshold','spots_found','max_intensity','no.frames','referenceSpot_Alpha','referenceSpot_gamma','beta','gamma','numberSpotmin'])
                df = pd.concat([df,df2])
#             df = df.sort_values(by=['Movie_Name']) 
            except ValueError:
                pass

0
cell number : cell_1
selected threshold :  600.0
path_in/out: C:/Users/uid-1204/Desktop/test2_bigFISHLIVE/Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_1
Total number of images : 166
image list found!
Found 865
Found 865 spots, max intensity = 2717


C:\Users\uid-1204\AppData\Local\anaconda3\envs\bigfishLive\lib\site-packages\bigfish\detection\spot_detection.py:586: RuntimeWarning: divide by zero encountered in log
  count_spots = np.log([np.count_nonzero(value_spots > t)
C:\Users\uid-1204\AppData\Local\anaconda3\envs\bigfishLive\lib\site-packages\bigfish\detection\spot_detection.py:586: RuntimeWarning: divide by zero encountered in log
  count_spots = np.log([np.count_nonzero(value_spots > t)
C:\Users\uid-1204\AppData\Local\anaconda3\envs\bigfishLive\lib\site-packages\bigfish\detection\spot_detection.py:586: RuntimeWarning: divide by zero encountered in log
  count_spots = np.log([np.count_nonzero(value_spots > t)
C:\Users\uid-1204\AppData\Local\anaconda3\envs\bigfishLive\lib\site-packages\bigfish\detection\spot_detection.py:586: RuntimeWarning: divide by zero encountered in log
  count_spots = np.log([np.count_nonzero(value_spots > t)
C:\Users\uid-1204\AppData\Local\anaconda3\envs\bigfishLive\lib\site-packages\bigfish\detection\s

done!
1
cell number : cell_10
selected threshold :  0.0
path_in/out: C:/Users/uid-1204/Desktop/test2_bigFISHLIVE/Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_10
2
cell number : cell_11
selected threshold :  0.0
path_in/out: C:/Users/uid-1204/Desktop/test2_bigFISHLIVE/Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_11
3
cell number : cell_12
selected threshold :  600.0
path_in/out: C:/Users/uid-1204/Desktop/test2_bigFISHLIVE/Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_12


C:\Users\uid-1204\AppData\Local\anaconda3\envs\bigfishLive\lib\site-packages\bigfish\detection\spot_detection.py:586: RuntimeWarning: divide by zero encountered in log
  count_spots = np.log([np.count_nonzero(value_spots > t)


Total number of images : 180
image list found!
Found 11344
Found 11344 spots, max intensity = 2942


...

done!
13
cell number : cell_6
selected threshold :  0.0
path_in/out: C:/Users/uid-1204/Desktop/test2_bigFISHLIVE/Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_6
14
cell number : cell_7
selected threshold :  0.0
path_in/out: C:/Users/uid-1204/Desktop/test2_bigFISHLIVE/Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_7
15
cell number : cell_8
selected threshold :  700.0
path_in/out: C:/Users/uid-1204/Desktop/test2_bigFISHLIVE/Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_8
Total number of images : 180
image list found!
Found 22674
Found 22674 spots, max intensity = 5698


C:\Users\uid-1204\AppData\Local\anaconda3\envs\bigfishLive\lib\site-packages\bigfish\detection\spot_detection.py:586: RuntimeWarning: divide by zero encountered in log
  count_spots = np.log([np.count_nonzero(value_spots > t)


done!
16
cell number : cell_9
selected threshold :  550.0
path_in/out: C:/Users/uid-1204/Desktop/test2_bigFISHLIVE/Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_9
Total number of images : 180
image list found!
Found 16552
Found 16552 spots, max intensity = 2762


C:\Users\uid-1204\AppData\Local\anaconda3\envs\bigfishLive\lib\site-packages\bigfish\detection\spot_detection.py:586: RuntimeWarning: divide by zero encountered in log
  count_spots = np.log([np.count_nonzero(value_spots > t)
C:\Users\uid-1204\AppData\Local\anaconda3\envs\bigfishLive\lib\site-packages\bigfish\detection\spot_detection.py:586: RuntimeWarning: divide by zero encountered in log
  count_spots = np.log([np.count_nonzero(value_spots > t)
C:\Users\uid-1204\AppData\Local\anaconda3\envs\bigfishLive\lib\site-packages\bigfish\detection\spot_detection.py:586: RuntimeWarning: divide by zero encountered in log
  count_spots = np.log([np.count_nonzero(value_spots > t)
C:\Users\uid-1204\AppData\Local\anaconda3\envs\bigfishLive\lib\site-packages\bigfish\detection\spot_detection.py:586: RuntimeWarning: divide by zero encountered in log
  count_spots = np.log([np.count_nonzero(value_spots > t)


done!


In [24]:
# df = df.sort_values(by=['Movie_Name'])
df.to_csv(baseFolder0+'/spots&ClustersSummary_27_May_2025.csv') 

In [25]:
df

,imsq,cell_number,threshold,spots_found,max_intensity,no.frames,referenceSpot_Alpha,referenceSpot_gamma,beta,gamma,numberSpotmin
0,06,cell_1,600.0,865,2717,166,0.6,3,1,3,2
0,06,cell_12,600.0,11344,2942,180,0.6,3,1,3,2
0,06,cell_14,620.0,4938,2836,180,0.6,3,1,3,2
0,06,cell_16,598.0,24178,4200,180,0.6,3,1,3,2
0,06,cell_18,598.0,14439,4862,180,0.6,3,1,3,2
0,06,cell_2,830.0,2909,3886,180,0.6,3,1,3,2
0,06,cell_3,480.0,1439,1951,180,0.6,3,1,3,2
0,06,cell_4,708.0,24560,5812,180,0.6,3,1,3,2
0,06,cell_5,680.0,1470,3127,180,0.6,3,1,3,2
0,06,cell_8,700.0,22674,5698,180,0.6,3,1,3,2
